In [4]:
import os
import json
import glob
import torch
import torch.nn.functional as F
import pickle
import numpy as np
from utils.HICO_utils import calc_ap, get_map, hoi_no_inter_all

In [6]:
res = pickle.load(open("./DATA/hico_clip/result_0.pkl", "rb"))
keys = np.array(res["keys"])
scores = np.array(res["probs"])
scores = torch.sigmoid(torch.from_numpy(scores/10)).numpy()

A2_dir = "A2_generate_rule_20230514"
B1_dir = "B1_tgt_keys_clip-zs"
B2_dir = "B2_recog_entity_clip-zs_20230514"

fac_dict = pickle.load(open("./DATA/fac_hico_clip+reason.pkl", "rb"))
missing_hoi_ids = []
map = np.zeros(600)
for hoi_id in range(600):
    
    print("hoi_id: ", hoi_id)
    ap, _ = calc_ap(scores[:, hoi_id], keys, hoi_id)

    ## for "no_interaction" HOIs, we use the original results without the reasoning results
    if hoi_id + 1 in hoi_no_inter_all:
        map[hoi_id] = ap
        continue
    
    ## else, we use the original results plus the reasoning results
    tgt_keys = pickle.load(open("./DATA/%s/%d.pkl"%(B1_dir, hoi_id), "rb"))
    ap_list = []
    fac = fac_dict[hoi_id]
    cur_scores = scores[:, hoi_id].copy()
    
    for key in tgt_keys:
        pred_entities = pickle.load(open("./DATA/%s/%d/%s.pkl"%(B2_dir, hoi_id, key), "rb"))
        if len(pred_entities) == 0:
            continue

        pred_entities = torch.from_numpy(np.array(pred_entities))
        pred_entities = F.softmax(pred_entities, dim=-1)[:, 1]
        pred_entities = pred_entities.numpy()

        hoi_entities = json.load(open("./DATA/%s/%d/hoi_entities.json"%(A2_dir, hoi_id), "rb"))
        hoi_rules = json.load(open("./DATA/%s/%d/hoi_rule.json"%(A2_dir, hoi_id), "rb"))
        pred_rules = []
        for i, rule in enumerate(hoi_rules):
            if rule[0] == 1:
                pred_rule = [pred_entities[hoi_entities[ent]] for ent in rule[1:]]
                pred_rule = np.array(pred_rule).min()
                pred_rules.append(pred_rule)
        pred_rules = np.array(pred_rules)
        pred_rules = pred_rules.max()
        index = np.where(keys==key)[0][0]
        cur_scores[index] = cur_scores[index] + fac * pred_rules + 100

    ap, _ = calc_ap(cur_scores, keys, hoi_id)
    map[hoi_id] = ap
    print("ap:", np.round(100*ap, 2))
    print('-----------')
print("mAP: ", np.round(100*map.mean(), 2))

hoi_id:  0
ap: 34.23
-----------
hoi_id:  1
ap: 41.9
-----------
hoi_id:  2
ap: 5.0
-----------
hoi_id:  3
ap: 58.54
-----------
hoi_id:  4
ap: 53.32
-----------
hoi_id:  5
ap: 29.79
-----------
hoi_id:  6
ap: 17.29
-----------
hoi_id:  7
ap: 10.16
-----------
hoi_id:  8
ap: 78.33
-----------
hoi_id:  9
hoi_id:  10
ap: 6.74
-----------
hoi_id:  11
ap: 53.64
-----------
hoi_id:  12
ap: 50.42
-----------
hoi_id:  13
ap: 77.54
-----------
hoi_id:  14
ap: 2.0
-----------
hoi_id:  15
ap: 4.1
-----------
hoi_id:  16
ap: 12.72
-----------
hoi_id:  17
ap: 68.34
-----------
hoi_id:  18
ap: 55.44
-----------
hoi_id:  19
ap: 51.28
-----------
hoi_id:  20
ap: 58.07
-----------
hoi_id:  21
ap: 9.98
-----------
hoi_id:  22
ap: 72.73
-----------
hoi_id:  23
hoi_id:  24
ap: 67.35
-----------
hoi_id:  25
ap: 85.71
-----------
hoi_id:  26
ap: 59.62
-----------
hoi_id:  27
ap: 34.72
-----------
hoi_id:  28
ap: 34.17
-----------
hoi_id:  29
ap: 48.38
-----------
hoi_id:  30
hoi_id:  31
ap: 21.06
---------

In [7]:
print("mAP: ", np.round(100*map.mean(), 2))

mAP:  43.21
